In [1]:
import numpy as np
import pandas as pd
import cv2
import PIL
import pytesseract

image = cv2.imread("ic/Azam v3.jpg")

def base_All(image):
    x = 15
    y = 120
    w = 960
    h = 485
    cv2.rectangle(image, (x,y),(x+w, y+h), (36,255,12),1)
    cropped_image = image[y:y+h, x:x+w]
    cv2.imwrite('contour.png', cropped_image )
    return image

cropped_image = cv2.imread("contour.png")
cv2.imwrite("temp/all.png",base_All(image))

gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
cv2.imwrite("temp/gray.jpg", gray)

blur = cv2.GaussianBlur(gray, (7,7),0)
cv2.imwrite("temp/blur.jpg", blur)

thresh, im_bw = cv2.threshold(gray, 150, 250, cv2.THRESH_BINARY)
cv2.imwrite("temp/thresh.jpg", thresh)
cv2.imwrite("temp/bw.jpg", im_bw)

inverted = cv2.bitwise_not(im_bw)
cv2.imwrite("temp/inverted.jpg", inverted)

def thin_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernel = np.ones((3,2), np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return(image)

eroded_image = thin_font(im_bw)
cv2.imwrite("temp/eroded_image.jpg", eroded_image)

def thick_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernel = np.ones((3,2), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return(image)

dilated_image = thick_font(eroded_image)
cv2.imwrite("temp/dilated_image.jpg", dilated_image)

def noise_removal(image):
    import numpy as np
    kernel = np.ones((1,1), np.uint8)
    image = cv2.dilate(image, kernel, iterations=2)
    kernel = np.ones((1,1), np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    image = cv2.medianBlur(image, 3)
    return(image)

no_noise = noise_removal(dilated_image)
cv2.imwrite("temp/no_noise.jpg", no_noise)

final_image = thin_font(no_noise)
cv2.imwrite("temp/final_image.jpg", final_image)

#text_cv = pytesseract.image_to_string(thin_font2)
#print(text_cv)

data = pytesseract.image_to_data(final_image)
dataList = list(map(lambda x: x.split('\t'),data.split('\n')))
df = pd.DataFrame(dataList[1:],columns=dataList[0])
df.head(10)




,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,960,485,-1,
1,2,1,1,0,0,0,15,12,506,73,-1,
2,3,1,1,1,0,0,15,12,506,73,-1,
3,4,1,1,1,1,0,15,12,506,73,-1,
4,5,1,1,1,1,1,15,14,283,32,78,990106-08-685
5,5,1,1,1,1,2,305,16,17,30,96,7
6,5,1,1,1,1,3,486,12,35,73,38,Ih
7,2,1,2,0,0,0,71,73,164,151,-1,
8,3,1,2,1,0,0,71,73,164,151,-1,
9,4,1,2,1,1,0,71,73,164,151,-1,


In [2]:
df.dropna(inplace=True) # drop the missing in rows
col_int = ['level','page_num','block_num','par_num','line_num','word_num','left','top','width','height','conf']
df[col_int] = df[col_int].astype(int)

In [3]:
df.dtypes

level         int32
page_num      int32
block_num     int32
par_num       int32
line_num      int32
word_num      int32
left          int32
top           int32
width         int32
height        int32
conf          int32
text         object
dtype: object

In [4]:
cropped = cv2.imread("contour.png")

In [5]:

image2 = cropped.copy()

In [14]:
level = 'line'
for l,x,y,w,h,c,txt in df[['level','left','top','width','height','conf','text']].values:
    #print(l,x,y,w,h,c)
    if level == 'page':
        if l == 1:
            cv2.rectangle(image2,(x,y),(x+w,y+h),(0,0,0),2)
        else:
            continue
            
    elif level == 'block':
        if l == 2:
            cv2.rectangle(image2,(x,y),(x+w,y+h),(255,0,0),2)
        else:
            continue
    
    elif level == 'para':
        if l == 3:
            cv2.rectangle(image2,(x,y),(x+w,y+h),(0,255,0),2)
        else:
            continue
    
    elif level == 'line':
        if l == 4:
            cv2.rectangle(image2,(x,y),(x+w,y+h),(0,0,255),2)
        else:
            continue
            
    elif level == 'word':
        if l == 5:
            cv2.rectangle(image2,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(image2,txt,(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,0),2)
        else:
            continue
            
cv2.imshow("bounding box",image2)
cv2.waitKey()
cv2.destroyAllWindows()

In [7]:
import os
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')


In [8]:
imgPaths = glob('./cropped ic/*.png')

In [9]:
imgPaths

['./cropped ic\\Aiman.png',
 './cropped ic\\Azam.png',
 './cropped ic\\Fila.png',
 './cropped ic\\Hanis.png',
 './cropped ic\\Helmi.png']

Extract data and text

In [10]:
ICAll = pd.DataFrame(columns = ['id'])

for imgPath in tqdm(imgPaths,desc='ICALL'):
    
    #imgPath = imgPaths[0]
    _, filename = os.path.split(imgPath)
    
    #extract data and text
    image2 = cv2.imread(imgPath)
    data = pytesseract.image_to_data(image2)
    dataList = list(map(lambda x: x.split('\t'), data.split('\n')))
    df = pd.DataFrame(dataList[1:],columns = dataList[0])
    df.dropna(inplace=True) #drop the missing in rows
    df['conf'] = df['conf'].astype(int)

    useFulData = df.query('conf >= 30')
    
    # dataframe 
    IC = pd.DataFrame()
    IC['text'] = useFulData['text']
    IC['id'] = filename
    
    #concatenation
    ICAll = pd.concat((ICAll,IC))

ICALL: 100%|██████████| 5/5 [00:08<00:00,  1.76s/it]


In [11]:
df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,960,485,-1,
1,2,1,1,0,0,0,4,2,341,50,-1,
2,3,1,1,1,0,0,4,2,341,50,-1,
3,4,1,1,1,1,0,4,2,341,50,-1,
4,5,1,1,1,1,1,4,2,341,50,78,990914-08-5131
5,2,1,2,0,0,0,180,152,1,30,-1,
6,3,1,2,1,0,0,180,152,1,30,-1,
7,4,1,2,1,1,0,180,152,1,30,-1,
8,5,1,2,1,1,1,180,152,1,30,95,
9,2,1,3,0,0,0,72,73,166,154,-1,


In [12]:
ICAll.to_csv('ic.csv',index = False)

In [13]:
IC

,text,id
4,990914-08-5131,Helmi.png
8,,Helmi.png
12,,Helmi.png
16,NURHELM!,Helmi.png
17,HASHIMI,Helmi.png
18,BIN,Helmi.png
19,AHMAD,Helmi.png
21,NO,Helmi.png
22,132,Helmi.png
27,ae,Helmi.png
